In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

In [2]:
# Loading the datasets
data_train = np.load(r"Dataset/Trousers & Jeans - All - Train.npz")
data_val = np.load(r"Dataset/Trousers & Jeans - All - Validation.npz")
data_test = np.load(r"Dataset/Trousers & Jeans - All - Test.npz")

In [3]:
# Extracting the arrays from the imported data
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_val['images']
labels_val = data_val['labels']

images_test = data_test['images']
labels_test = data_test['labels']

In [4]:
# Scaling the pixel values of all images 
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

In [5]:
# Defining the hyperparameters we would tune, and their values to be tested
HP_FILTER_SIZE_1 = hp.HParam('filter_size_1', hp.Discrete([3,5,7]))
HP_FILTER_NUM = hp.HParam('filters_number', hp.Discrete([32,64,96,128]))
HP_FILTER_SIZE_2 = hp.HParam('filter_size_2', hp.Discrete([3,5]))

METRIC_ACCURACY = 'accuracy'

# Logging setup info
with tf.summary.create_file_writer(r'Logs/Model 2/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE_1, HP_FILTER_NUM, HP_FILTER_SIZE_2],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [6]:
EPOCHS = 20
BATCH_SIZE = 64

In [9]:
# Wrapping our model and training in a function
def train_test_model(hparams, session_num):
    
    # Outlining the model/architecture of our CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], hparams[HP_FILTER_SIZE_1], activation='relu', input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], hparams[HP_FILTER_SIZE_2], activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4)
    ])
    
    # Defining the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Compiling the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

    # Defining the logging directory
    log_dir = "Logs/Model 2/fit/" + "run-{}".format(session_num)
    
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
          cm (array, shape = [n, n]): a confusion matrix of integer classes
          class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure 
    
    
    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image
    
    
    
    # Defining a file writer for Confusion Matrix logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')
    
    
    
    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Trousers Male', 'Jeans Male', 'Trousers Female', "Jeans Female"])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    
    # Define the Tensorboard and Confusion Matrix callbacks.
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val,labels_val),
        verbose = 2
    )
    
    # Evaluating the model's performance on the validation set
    _, accuracy = model.evaluate(images_val,labels_val)
    
    # Saving the current model for future reference
    model.save(r"saved_models/Model 2/Run-{}".format(session_num))
    
    return accuracy

In [10]:
# Creating a function to log the resuls
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [11]:
session_num = 1

for filter_size_1 in HP_FILTER_SIZE_1.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:
        for filter_size_2 in HP_FILTER_SIZE_2.domain.values:

            hparams = {
                HP_FILTER_SIZE_1: filter_size_1,
                HP_FILTER_NUM: filter_num,
                HP_FILTER_SIZE_2: filter_size_2
            }

            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('Logs/Model 2/hparam_tuning/' + run_name, hparams, session_num)

            session_num += 1

--- Starting trial: run-1
{'filter_size_1': 3, 'filters_number': 32, 'filter_size_2': 3}
Epoch 1/20
63/63 - 31s - loss: 1.1792 - accuracy: 0.4833 - val_loss: 1.0554 - val_accuracy: 0.5540
Epoch 2/20
63/63 - 31s - loss: 0.8927 - accuracy: 0.6379 - val_loss: 0.9067 - val_accuracy: 0.6400
Epoch 3/20
63/63 - 34s - loss: 0.7886 - accuracy: 0.6836 - val_loss: 0.8695 - val_accuracy: 0.6220
Epoch 4/20
63/63 - 32s - loss: 0.7532 - accuracy: 0.6948 - val_loss: 0.8337 - val_accuracy: 0.6780
Epoch 5/20
63/63 - 31s - loss: 0.6856 - accuracy: 0.7350 - val_loss: 0.7774 - val_accuracy: 0.6960
Epoch 6/20
63/63 - 35s - loss: 0.6458 - accuracy: 0.7510 - val_loss: 0.8208 - val_accuracy: 0.7160
Epoch 7/20
63/63 - 36s - loss: 0.6518 - accuracy: 0.7428 - val_loss: 0.8987 - val_accuracy: 0.6420
16/16 [==============================] - 1s 83ms/step - loss: 0.7774 - accuracy: 0.6960
INFO:tensorflow:Assets written to: saved_models/Model 2/Run-1/assets
--- Starting trial: run-2
{'filter_size_1': 3, 'filters_numbe

Epoch 1/20
63/63 - 278s - loss: 1.2532 - accuracy: 0.4800 - val_loss: 1.1193 - val_accuracy: 0.5320
Epoch 2/20
63/63 - 297s - loss: 0.9362 - accuracy: 0.6161 - val_loss: 1.0079 - val_accuracy: 0.6080
Epoch 3/20
63/63 - 299s - loss: 0.8408 - accuracy: 0.6676 - val_loss: 0.9173 - val_accuracy: 0.6300
Epoch 4/20
63/63 - 286s - loss: 0.7635 - accuracy: 0.6846 - val_loss: 0.8807 - val_accuracy: 0.6400
Epoch 5/20
63/63 - 452s - loss: 0.7005 - accuracy: 0.7180 - val_loss: 0.8192 - val_accuracy: 0.6900
Epoch 6/20
63/63 - 314s - loss: 0.6664 - accuracy: 0.7273 - val_loss: 1.1920 - val_accuracy: 0.6000
Epoch 7/20
63/63 - 325s - loss: 0.6238 - accuracy: 0.7488 - val_loss: 0.8112 - val_accuracy: 0.6900
Epoch 8/20
63/63 - 312s - loss: 0.5870 - accuracy: 0.7745 - val_loss: 0.7834 - val_accuracy: 0.7060
Epoch 9/20
63/63 - 295s - loss: 0.5388 - accuracy: 0.7950 - val_loss: 0.8487 - val_accuracy: 0.6740
Epoch 10/20
63/63 - 277s - loss: 0.5282 - accuracy: 0.7982 - val_loss: 0.8553 - val_accuracy: 0.7020

Epoch 4/20
63/63 - 1011s - loss: 0.7232 - accuracy: 0.7015 - val_loss: 0.8489 - val_accuracy: 0.6800
Epoch 5/20
63/63 - 205s - loss: 0.6920 - accuracy: 0.7268 - val_loss: 0.8352 - val_accuracy: 0.6500
Epoch 6/20
63/63 - 182s - loss: 0.6505 - accuracy: 0.7428 - val_loss: 0.7843 - val_accuracy: 0.7060
Epoch 7/20
63/63 - 2021s - loss: 0.6228 - accuracy: 0.7500 - val_loss: 0.7387 - val_accuracy: 0.7040
Epoch 8/20
63/63 - 212s - loss: 0.5543 - accuracy: 0.7852 - val_loss: 0.8337 - val_accuracy: 0.7140
Epoch 9/20
63/63 - 228s - loss: 0.5303 - accuracy: 0.7912 - val_loss: 0.8295 - val_accuracy: 0.6900
16/16 [==============================] - 8s 479ms/step - loss: 0.7387 - accuracy: 0.7040
INFO:tensorflow:Assets written to: saved_models/Model 2/Run-14/assets
--- Starting trial: run-15
{'filter_size_1': 5, 'filters_number': 128, 'filter_size_2': 3}
Epoch 1/20
63/63 - 207s - loss: 1.2286 - accuracy: 0.4735 - val_loss: 1.0125 - val_accuracy: 0.5980
Epoch 2/20
63/63 - 202s - loss: 0.8971 - accurac

63/63 - 112s - loss: 1.2076 - accuracy: 0.4675 - val_loss: 1.3442 - val_accuracy: 0.5080
Epoch 2/20
63/63 - 109s - loss: 0.9381 - accuracy: 0.6139 - val_loss: 0.9562 - val_accuracy: 0.5940
Epoch 3/20
63/63 - 112s - loss: 0.8510 - accuracy: 0.6536 - val_loss: 0.8982 - val_accuracy: 0.6440
Epoch 4/20
63/63 - 115s - loss: 0.7799 - accuracy: 0.6848 - val_loss: 0.8630 - val_accuracy: 0.6680
Epoch 5/20
63/63 - 110s - loss: 0.7286 - accuracy: 0.7018 - val_loss: 0.8478 - val_accuracy: 0.6720
Epoch 6/20
63/63 - 113s - loss: 0.7037 - accuracy: 0.7198 - val_loss: 0.8638 - val_accuracy: 0.6500
Epoch 7/20
63/63 - 110s - loss: 0.6751 - accuracy: 0.7258 - val_loss: 0.8689 - val_accuracy: 0.6580
16/16 [==============================] - 3s 202ms/step - loss: 0.8478 - accuracy: 0.6720
INFO:tensorflow:Assets written to: saved_models/Model 2/Run-20/assets
--- Starting trial: run-21
{'filter_size_1': 7, 'filters_number': 96, 'filter_size_2': 3}
Epoch 1/20
63/63 - 114s - loss: 1.2493 - accuracy: 0.4503 - va

In [12]:
# Loading a model to evaluate on the test set
model = tf.keras.models.load_model(r"saved_models/Model 2/Run-1")

In [13]:
test_loss, test_accuracy = model.evaluate(images_test,labels_test)

16/16 [==============================] - 2s 64ms/step - loss: 0.8034 - accuracy: 0.7080


In [14]:
# Printing the test results
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.8034. Test accuracy: 70.80%


In [16]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 2/hparam_tuning"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6013 (pid 15483), started 0:08:12 ago. (Use '!kill 15483' to kill it.)

In [17]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 2/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 15906.